# DSCI: 525 Milestone 1 - Group 8

### Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

C:\Users\SantiagoRugelesSch\miniconda3\envs\525\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


### Downloading the data

In [3]:
%pwd

'C:\\Users\\SantiagoRugelesSch\\DSCI_525\\525-group8\\notebooks'

In [4]:
%cd ~/DSCI_525/525-group8

C:\Users\SantiagoRugelesSch\DSCI_525\525-group8


In [13]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "dailyrainfall/"

In [14]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [15]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

Wall time: 2min 36s


In [16]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

Wall time: 1min 27s


### Combining data CSVs

In [5]:
#Test to check the command 
use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
BCC_df = pd.read_csv("dailyrainfall/BCC-CSM2-MR_daily_rainfall_NSW.csv", usecols=use_cols, parse_dates=["time"])
BCC_df

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-35.887649,-34.766162,141.1875,142.3125,4.265520e-17
1,1889-01-02 12:00:00,-35.887649,-34.766162,141.1875,142.3125,2.728501e-20
2,1889-01-03 12:00:00,-35.887649,-34.766162,141.1875,142.3125,5.221255e-24
3,1889-01-04 12:00:00,-35.887649,-34.766162,141.1875,142.3125,2.869350e-17
4,1889-01-05 12:00:00,-35.887649,-34.766162,141.1875,142.3125,2.290091e-15
...,...,...,...,...,...,...
3035335,2014-12-27 12:00:00,-30.280211,-29.158723,152.4375,153.5625,1.218327e+01
3035336,2014-12-28 12:00:00,-30.280211,-29.158723,152.4375,153.5625,6.209972e-01
3035337,2014-12-29 12:00:00,-30.280211,-29.158723,152.4375,153.5625,3.562820e-04
3035338,2014-12-30 12:00:00,-30.280211,-29.158723,152.4375,153.5625,4.175968e+01


In [13]:
%%time
%memit
# Join all data together
#
import pandas as pd
use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
files = glob.glob('dailyrainfall/*.csv')
df = pd.DataFrame(columns = use_cols)
for file in files:
    if len(re.findall(r'NSW', file)) > 0: # Used to ignore rainfal SYD as it misses most columns
        print(file)
        df2 = (pd.read_csv(file,usecols=use_cols, parse_dates=["time"]).
               assign(model=re.findall(r'\\(.+?)_', file)[0]))
        df = pd.concat([df, df2], axis=0)



peak memory: 1245.37 MiB, increment: 0.61 MiB
dailyrainfall\ACCESS-CM2_daily_rainfall_NSW.csv
dailyrainfall\ACCESS-ESM1-5_daily_rainfall_NSW.csv
dailyrainfall\AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
dailyrainfall\BCC-CSM2-MR_daily_rainfall_NSW.csv
dailyrainfall\BCC-ESM1_daily_rainfall_NSW.csv
dailyrainfall\CanESM5_daily_rainfall_NSW.csv
dailyrainfall\CMCC-CM2-HR4_daily_rainfall_NSW.csv
dailyrainfall\CMCC-CM2-SR5_daily_rainfall_NSW.csv
dailyrainfall\CMCC-ESM2_daily_rainfall_NSW.csv
dailyrainfall\EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
dailyrainfall\FGOALS-f3-L_daily_rainfall_NSW.csv
dailyrainfall\FGOALS-g3_daily_rainfall_NSW.csv
dailyrainfall\GFDL-CM4_daily_rainfall_NSW.csv
dailyrainfall\GFDL-ESM4_daily_rainfall_NSW.csv
dailyrainfall\INM-CM4-8_daily_rainfall_NSW.csv
dailyrainfall\INM-CM5-0_daily_rainfall_NSW.csv
dailyrainfall\KIOST-ESM_daily_rainfall_NSW.csv
dailyrainfall\MIROC6_daily_rainfall_NSW.csv
dailyrainfall\MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
dailyrainfall\MPI-ESM1-2-HR_daily_r

In [14]:
%%time
%memit
df.to_csv("dailyrainfall/combined_rainfall_data.csv")

peak memory: 1714.43 MiB, increment: 1.20 MiB
Wall time: 16min 13s


### Load the combined CSV to memory and perform a simple EDA

In [8]:
%%time
%memit
rainfall_df = pd.read_csv("dailyrainfall/combined_rainfall_data.csv", index_col=0, parse_dates=["time"])
rainfall_df.sample(15)

peak memory: 1804.73 MiB, increment: 1.76 MiB


C:\Users\SantiagoRugelesSch\miniconda3\envs\525\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Wall time: 5min 56s


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
2306018,1902-09-29 12:00:00,-31.401700,-30.280215,147.93750,149.06250,0.000185,MRI-ESM2-0
2793195,1981-08-04 12:00:00,-29.900000,-29.100000,141.47500,142.27500,7.077067,FGOALS-f3-L
2842562,1990-11-04 12:00:00,-31.099476,-30.157068,148.12500,149.37500,4.153233,NorESM2-MM
2044755,1947-01-26 12:00:00,-32.041885,-31.099476,140.62500,141.87500,47.186990,TaiESM1
1091619,1981-09-27 12:00:00,-34.000000,-33.000000,145.00000,146.25000,0.005237,GFDL-CM4
225860,2003-10-18 12:00:00,-36.455696,-34.430380,149.00000,151.00000,7.030256,FGOALS-g3
766330,1972-07-15 12:00:00,-33.000000,-31.500000,145.00000,147.00000,2.309932,INM-CM4-8
283213,1908-06-04 12:00:00,-35.439867,-33.574619,152.81250,154.68750,0.264943,MPI-ESM1-2-LR
2009545,1976-08-09 12:00:00,-32.984293,-32.041885,153.12500,154.37500,0.306905,CMCC-ESM2
374255,1906-05-11 12:00:00,-35.811518,-34.869110,150.62500,151.87500,6.232773,CMCC-CM2-HR4


In [10]:
rainfall_df.head(15)

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2
5,1889-01-06 12:00:00,-36.25,-35.0,140.625,142.5,3.292260e-02,ACCESS-CM2
6,1889-01-07 12:00:00,-36.25,-35.0,140.625,142.5,8.906468e-02,ACCESS-CM2
7,1889-01-08 12:00:00,-36.25,-35.0,140.625,142.5,3.158012e-02,ACCESS-CM2
8,1889-01-09 12:00:00,-36.25,-35.0,140.625,142.5,3.107391e-02,ACCESS-CM2
9,1889-01-10 12:00:00,-36.25,-35.0,140.625,142.5,3.304103e-02,ACCESS-CM2


In [14]:
%%time
%memit
rainfall_df.shape

peak memory: 1565.47 MiB, increment: 0.02 MiB
Wall time: 8.74 s


(62467843, 7)

In [15]:
%%time
%memit
rainfall_df.describe()

peak memory: 1565.49 MiB, increment: -0.00 MiB
Wall time: 56 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901170e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [16]:
rainfall_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62467843 entries, 0 to 3541229
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   time           datetime64[ns]
 1   lat_min        float64       
 2   lat_max        float64       
 3   lon_min        float64       
 4   lon_max        float64       
 5   rain (mm/day)  float64       
 6   model          object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 3.7+ GB


### Perform a simple EDA in R

In [6]:
# Rachel and Rui